# Star Processing

In [ ]:
import numpy as np, torch, torchvision, torchvision.transforms as T, matplotlib.pyplot as plt, random as rr, copy
from PIL import Image

def load_image(path):
    img = Image.open(path)
    # img = T.Resize((224, 224))(img)
    img = T.ToTensor()(img) # But this leads to loss of information, so try this:

    # img = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img)
    # Convert to grey scale by averaging the 3 channels
    # img = img.mean(0)
    # Convert to binary file with 0 and 255 based on a threshold = 0.7
    # img = (img > 0.7).float() * 255
    
    # img = img.unsqueeze(0)
    return img

# Important
def max_pool_image(img, kernel_size=2, stride=2):
    img = torch.nn.functional.max_pool2d(img, kernel_size=kernel_size, stride=stride)
    return img

def binarize_image(img, threshold=0.8):
    img = (img > threshold).float() * 255
    return img

# Less Important
def avg_pool_image(img, kernel_size=2, stride=2):
    img = torch.nn.functional.avg_pool2d(img, kernel_size=kernel_size, stride=stride)
    return img
def upsample_image(img, scale_factor=2): # It is also called deconvolution - it is the opposite of max_pooling. It will increase the size of the image
    img = torch.nn.functional.interpolate(img, scale_factor=scale_factor, mode='nearest')
    return img
def downsample_image(img, scale_factor=2): # It will reduce the size of the 
    img = torch.nn.functional.interpolate(img, scale_factor=1/scale_factor, mode='nearest')
    return img
def convolve_image(img, kernel):
    img = torch.nn.functional.conv2d(img, kernel)
    return img
def normalize_image(img, mean, std):
    img = T.Normalize(mean=mean, std=std)(img)
    return img


def plot_image(img):
    plt.imshow(img[0], cmap='gray')
    plt.show()


In [ ]:
def pranav_camera(path):
    img = Image.open(path)
    img = T.ToTensor()(img)
    img = binarize_image(img, threshold=0.1)
    # img = avg_pool_image(img)
    img = max_pool_image(img)
    # img = binarize_image(img, threshold=0.7)
    return img

In [ ]:
def manas_camera(path):
    img = Image.open(path)
    img = T.ToTensor()(img)
    img = binarize_image(img, threshold=0.4)
    img = max_pool_image(img)
    img = max_pool_image(img)
    img = max_pool_image(img)
    # img = binarize_image(img, threshold=0.7)
    img = img.int()
    img = img[0]|img[1]|img[2]
    return img

In [ ]:
def pranav_camera(path):
    img = Image.open(path)
    img = T.ToTensor()(img)
    img = binarize_image(img, threshold=0.1)
    img = max_pool_image(img)
    img = max_pool_image(img)
    img = max_pool_image(img)
    img = binarize_image(img, threshold=0.7)
    img = img.int()
    img = img[0] | img[1] | img[2]
    return img


In [ ]:

def get_clusters(img, i, j):
    if i < 0 or j < 0 or i >= img.shape[0] or j >= img.shape[1] or img[i][j] == 0:
        return (0, 0, 0)
    img[i][j] = 0
    x1, y1, c1 = get_clusters(img, i+1, j)
    x2, y2, c2 = get_clusters(img, i-1, j)
    x3, y3, c3 = get_clusters(img, i, j+1)
    x4, y4, c4 = get_clusters(img, i, j-1)
    x5, y5, c5 = get_clusters(img, i+1, j+1)
    x6, y6, c6 = get_clusters(img, i-1, j-1)
    x7, y7, c7 = get_clusters(img, i+1, j-1)
    x8, y8, c8 = get_clusters(img, i-1, j+1)
    return (i + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8, j + y1 + y2 + y3 + y4 + y5 + y6 + y7 + y8, c1 + c2 + c3 + c4 + c5 + c6 + c7 + c8 + 1) # x, y, count
# Let's first create a function to get the coordinates of the stars in the image.
def get_stars(img):
    # Let's create a recursive function to get the coordinates of the stars in the image.
    stars = []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            x, y, c = get_clusters(img, i, j)
            if c > 0:
                # stars.append(((x/c).__round__(3), (y/c).__round__(3), c))
                stars.append([(x/c).__round__(3), (y/c).__round__(3)])
    return stars


# path = f'skyimages/image2.jpg'
# img = pranav_camera(path)
# coordinaates = get_stars(copy.deepcopy(img))
# print('Coordinates of stars:', coordinaates)
# # Now, let's plot the stars in the image
# plt.imshow(img, cmap='gray')
# for star in coordinaates:
#     plt.scatter(star[1], star[0], c='red', s = 10)
# plt.show()


In [ ]:
def plot_stars(img):
    # find the coordinates of the stars
    img_temp = copy.deepcopy(img)
    coordinaates = get_stars(img_temp)
    # Now, let's plot the stars in the image
    print('Number of stars:', len(coordinaates))
    print('Coordinates of stars:', coordinaates)
    plt.imshow(img, cmap='gray')
    
    for star in coordinaates:
        plt.scatter(star[1], star[0], c='red', s = 1)
    plt.show()
    


Number of stars: 14  

**Traditional Method**  
8281  
Redundunt: 4792  

**Our Method**  
Interations: 1365  
Redundunt: 605  

In [ ]:
img = pranav_camera('skyimages/image5.jpg')
plt.imshow(img, cmap='gray')
plt.show()
# Base Algorithm 1 - Inefficient one
coord = get_stars(copy.deepcopy(img))
print(coord )
## Preprocessing
x_coord = [c[1].__round__(3) for c in coord]
y_coord = [c[0].__round__(3) for c in coord]
x_to_y = {int(x*1000):y for y,x in coord}
# print(x_coord, y_coord)
xs = list(sorted(x_coord))
ys = list(sorted(y_coord))
ind_ys = {int(y*1000):i for i,y in enumerate(ys)}
print(x_to_y)
print(ind_ys)
for x,y in zip(x_coord, y_coord):
    # plot vertical and horizontal lines
    plt.plot([x, x], [ys[0], ys[-1]], 'k-', lw=0.5)
    plt.plot([xs[0], xs[-1]], [y, y], 'k-', lw=0.5)
plt.scatter(x_coord, y_coord, c='red', s = 10)
plt.show()
N = len(xs)
print(f'Number of stars: {len(coord)}')

# Get through all sub regions
reg  = []
redundunt = 0
for xi in range(N):
    for xj in range(xi+1, N):
        ymin, ymax = sorted([x_to_y[int(x*1000)] for x in [xs[xi], xs[xj]]])
        yminind, ymaxind = ind_ys[int(ymin*1000)], ind_ys[int(ymax*1000)]

        for yi in range(yminind, ymaxind):
            for yj in range(yi+1, ymaxind):
                count = 0
                for x,y in coord:
                    if xs[xi] <= x <= xs[xj] and ys[yi] <= y <= ys[yj]:
                        count += 1
                if count > 0:
                    redundunt += 1
                reg.append((xs[xi], xs[xj], ys[yi], ys[yj], count))

print(len(reg))
# print(reg)
print(f'Redundunt: {redundunt}')
print(f'Percent Redundancy: {redundunt/len(reg)*100}%')


# Base Algorithm 2
reg  = []
redundunt = 0
for xi in range(N):
    for xj in range(xi+1, N):
        # ymin, ymax = sorted([x_to_y[int(x*1000)] for x in [xs[xi], xs[xj]]])
        # yminind, ymaxind = ind_ys[int(ymin*1000)], ind_ys[int(ymax*1000)]

        for yi in range(N):
            for yj in range(yi+1, N):
                count = 0
                for x,y in coord:
                    if xs[xi] <= x <= xs[xj] and ys[yi] <= y <= ys[yj]:
                        count += 1
                if count > 0:
                    redundunt += 1
                reg.append((xs[xi], xs[xj], ys[yi], ys[yj], count))
print(len(reg))
# print(reg)
print(f'Redundunt: {redundunt}')
print(f'Percent Redundancy: {redundunt/len(reg)*100}%')


In [ ]:
def print_subregions(coord: list[int]):
  """
  Prints all continuous subregions of stars in the given list.

  Args:
      coord: A list of 2D coordinates (x, y) as tuples.
  """
  # Create a dictionary where keys are coordinates rounded to the nearest thousandth,
  # and values are lists of stars in that coordinate.
  # choose ith star and jth star and find the region between them
  # Number of sub-regions = (n-1)C2
  num_subregions = 0
  constelations = {}
  for i in range(len(coord)):
    stars = [coord[i]]
    size = 1
    for j in range(i+1, len(coord)):
      stars.append(coord[j])
      size += 1
      plt.scatter([c[1] for c in stars], [c[0] for c in stars], c='green')
      print(f'Size: {size}')
      print(stars)
      # for con in constelations:
      #    if isequal1(con,stars):
      #       print(f"Detected a sub-region: {stars} as {con}")
      #       break
        # isequal1 -> change if basis
        # isequal2 -> angles
        # isequal3 -> normalized distances
      num_subregions += 1 
      # print(f"Sub-region {num_subregions}: {stars}")
  print(f"Number of sub-regions: {num_subregions}")

print_subregions(coord)

def plot_subregions(img):
    # find the coordinates of the stars
    img_temp = copy.deepcopy(img)
    coordinaates = get_stars(img_temp)
    # Now, let's plot the stars in the image
    print('Number of stars:', len(coordinaates))
    print('Coordinates of stars:', coordinaates)
    plt.imshow(img, cmap='gray')
    
    # for star in coordinaates:
    #     plt.scatter(star[1], star[0], c='red', s = 1)
    # plt.show()
    import matplotlib.patches as patches
    fig, ax = plt.subplots()
    ax.imshow(img, cmap='gray')
    for star in coordinaates:
        ax.scatter(star[1], star[0], c='red', s = 1)
    # Create a Rectangle patch
    for i in range(len(coordinaates)):
        s = 1
        x1, y1 = coordinaates[i][0], coordinaates[i][1]
        x2, y2 = coordinaates[i][0], coordinaates[i][1]
        for j in range(i+1, len(coordinaates)):
            s += 1
            x1, y1 = min(x1, coordinaates[j][0]), min(y1, coordinaates[j][1])
            x2, y2 = max(x2, coordinaates[j][0]), max(y2, coordinaates[j][1])
            rect = patches.Rectangle((y1, x1), y2-y1, x2-x1, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
    plt.show()
    

plot_subregions(img)

